In [1]:
import os
import esa_snappy
from esa_snappy import GPF
from esa_snappy import ProductIO, GeoPos, PixelPos, WKTReader
from esa_snappy import HashMap
from esa_snappy import jpy
import subprocess
from time import *
import numpy as np
from shapely.geometry import Point, Polygon
import geopandas as gpd
import sys
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from datetime import datetime

from snappy_functions import extract_info, temporal_baseline, perpendicular_baseline, read_product, write
from snappy_functions import get_subswath, check_same_subswath, topsar_split
from snappy_functions import apply_orbit_file, back_geocoding, enhanced_spectral_diversity
from snappy_functions import interferogram, topsar_deburst, goldstein_phase_filtering, subset
from snappy_functions import phase_to_elevation, terrain_correction
from snappy_functions import snaphu_export, snaphu_unwrapping, snaphu_import

# 1. Loading data

In [2]:
#a = r'_data\cabo_verde\small_perp_baseline\S1C_IW_SLC__1SDV_20250508T072956_20250508T073019_002236_004BF8_2AB6.SAFE' #cabo verde 1
#b = r'_data\cabo_verde\small_perp_baseline\S1C_IW_SLC__1SDV_20250520T072956_20250520T073020_002411_0050E8_4316.SAFE'# small baseline
#aoi = "POLYGON((-24.5393 14.7695,-24.2561 14.7695,-24.2561 15.0778,-24.5393 15.0778,-24.5393 14.7695))" 

a = r"_data\cabo_verde\big_perp_baseline\S1B_IW_SLC__1SDV_20210210T073014_20210210T073044_025541_030B1A_05A9.SAFE" #cabo verde 2
b = r"_data\cabo_verde\big_perp_baseline\S1B_IW_SLC__1SDV_20210306T073014_20210306T073044_025891_031686_A249.SAFE" #big baseline
aoi = "POLYGON((-24.5393 14.7695,-24.2561 14.7695,-24.2561 15.0778,-24.5393 15.0778,-24.5393 14.7695))"

#a = r"_data\costa_rica\small_perp_baseline\S1A_IW_SLC__1SDV_20250311T112253_20250311T112320_058256_073329_ED6E.SAFE" #costa rica 3
#b = r"_data\costa_rica\small_perp_baseline\S1A_IW_SLC__1SDV_20250323T112253_20250323T112320_058431_073A11_0FFE.SAFE" #small baseline
#aoi = "POLYGON((-83.4192 9.394,-82.9822 9.394,-82.9822 9.8997,-83.4192 9.8997,-83.4192 9.394))" 

#a = r"_data\costa_rica\big_perp_baseline\S1A_IW_SLC__1SDV_20250404T112254_20250404T112321_058606_074124_81E3.SAFE" #costa rica 4
#b = r"_data\costa_rica\big_perp_baseline\S1A_IW_SLC__1SDV_20250416T112253_20250416T112320_058781_07484E_810A.SAFE" #big baseline
#aoi = "POLYGON((-83.4192 9.394,-82.9822 9.394,-82.9822 9.8997,-83.4192 9.8997,-83.4192 9.394))" 

polarization = 'VV'

DEM = "SRTM 1Sec HGT (Auto Download)"
otherDEM = "Copernicus 30m Global DEM"

snaphu_export_xml_path = r"C:\Users\35196\repos\Sentinel1_DEMgeneration\_data\SnaphuExport.xml"
dim_snaphu_export=r"C:\Users\35196\repos\Sentinel1_DEMgeneration\_results\goldstein.dim"
target_folder_snaphu_export=r"C:\Users\35196\repos\Sentinel1_DEMgeneration\_results\_snaphu_export"

conf_file_path = r"C:\Users\35196\repos\Sentinel1_DEMgeneration\_results\_snaphu_export\goldstein\snaphu.conf"
snaphu_exe_path = r"C:\Users\35196\.snap\auxdata\snaphu-v2.0.4_win64\bin\snaphu.exe"
target_folder_snaphu_export_something_else = r"C:\Users\35196\repos\Sentinel1_DEMgeneration\_results\_snaphu_export\goldstein"

In [3]:
master = read_product(a)
slave = read_product(b)

# 2. Extracting basic information

## 2.1. Metadata information

In [6]:
extract_info(a)
extract_info(b)

Product name: S1B_IW_SLC__1SDV_20210210T073014_20210210T073044_025541_030B1A_05A9
Product type: SLC
Description: Sentinel-1 IW Level-1 SLC Product
Scene width: 68591
Scene height: 15060
Start time: 10-FEB-2021 07:30:14.834283
End time: 10-FEB-2021 07:30:44.654238

 Bands
Number of bands: 18
Band name: i_IW1_VH
Band name: q_IW1_VH
Band name: Intensity_IW1_VH
Band name: i_IW1_VV
Band name: q_IW1_VV
Band name: Intensity_IW1_VV
Band name: i_IW2_VH
Band name: q_IW2_VH
Band name: Intensity_IW2_VH
Band name: i_IW2_VV
Band name: q_IW2_VV
Band name: Intensity_IW2_VV
Band name: i_IW3_VH
Band name: q_IW3_VH
Band name: Intensity_IW3_VH
Band name: i_IW3_VV
Band name: q_IW3_VV
Band name: Intensity_IW3_VV


Product name: S1B_IW_SLC__1SDV_20210306T073014_20210306T073044_025891_031686_A249
Product type: SLC
Description: Sentinel-1 IW Level-1 SLC Product
Scene width: 68592
Scene height: 15060
Start time: 06-MAR-2021 07:30:14.418555
End time: 06-MAR-2021 07:30:44.238510

 Bands
Number of bands: 18
Band n

## 2.2. Temporal baseline

In [7]:
temporal_baseline(a, b)

Temporal Baseline: 24.0 days


# 3. Studying the Area of Interest

In [8]:
get_subswath (aoi, master)

'IW1'

In [9]:
get_subswath (aoi, slave)

'IW1'

In [10]:
check_same_subswath(master, slave, aoi)

Both product use the same subswath: IW1


# 4. Processing workflow

## 4.1. TOPSAR-Split

In [11]:
split_master = topsar_split(master, get_subswath(aoi, master), polarization, "1", "9")
split_slave = topsar_split(slave, get_subswath(aoi, slave), polarization, "1", "9")

Apply TOPSAR Split...
TOPSAR Split applied!
Apply TOPSAR Split...
TOPSAR Split applied!


In [12]:
write(split_master, '_results\split_master', 'BEAM-DIMAP')
write(split_slave, '_results\split_slave', 'BEAM-DIMAP')

Writing the product...
Product written!
Writing the product...
Product written!


## 4.2. Apply Orbit File

In [13]:
orbit_master = apply_orbit_file(read_product('_results\split_master.dim'))
orbit_slave = apply_orbit_file(read_product('_results\split_slave.dim'))

Apply Orbit File...
Orbit File applied!
Apply Orbit File...
Orbit File applied!


In [14]:
write(orbit_master, '_results\orbit_master', 'BEAM-DIMAP')
write(orbit_slave, '_results\orbit_slave', 'BEAM-DIMAP')

Writing the product...
Product written!
Writing the product...
Product written!


## 4.3. Back geocoding

In [15]:
back = back_geocoding([read_product('_results\orbit_master.dim'), read_product('_results\orbit_slave.dim')], DEM)

Back geocoding ...
Back geocoding applied!


In [16]:
write(back, r'_results\back_geocoding', 'BEAM-DIMAP')

Writing the product...
Product written!


## 4.4. Enhanced Spectral Diversity

In [17]:
enhanced = enhanced_spectral_diversity(read_product(r'_results\back_geocoding.dim'))

Enhancing Spectral Diversity ...
Enhanced Spectral Diversity applied!


In [18]:
write(enhanced, r'_results\enhanced', 'BEAM-DIMAP')

Writing the product...
Product written!


## 4.5. Interferogram Formation

In [19]:
interferogram = interferogram(read_product(r'_results\enhanced.dim'))

Creating interferogram ...
Interferogram created!


In [20]:
write(interferogram, r'_results\interferogram', 'BEAM-DIMAP')

Writing the product...
Product written!


## 4.6. Calculate perpendicular baseline

In [21]:
perpendicular_baseline(r'_results\interferogram.dim')

Perpendicular Baseline: 171.56344604492188 m


## 4.7. Deburst

In [22]:
deburst = topsar_deburst(read_product(r'_results\interferogram.dim'), polarization)

Apply TOPSAR Deburst...
TOPSAR Deburst applied!


In [23]:
write(deburst, r'_results\deburst', 'BEAM-DIMAP')

Writing the product...
Product written!


## 4.8. Goldstein Phase Filtering

In [24]:
goldstein = goldstein_phase_filtering(read_product(r'_results\deburst.dim'))

Apply Goldstein Phase Filtering...
Goldstein Phase Filtering applied!


In [25]:
write(goldstein, r'_results\goldstein', 'BEAM-DIMAP')

Writing the product...
Product written!


# 5. Snaphu process

## 5.1. Snaphu Export

In [4]:
snaphu_export(snaphu_export_xml_path, dim_snaphu_export, target_folder_snaphu_export)

Snaphu exporting...
Processing complete.
Output:
 Executing processing graph
....10%....20%....30%....40%....50%....60%....70%....80%....90% done.



## 5.2. Snaphu Unwrapping

In [ ]:
snaphu_unwrapping(conf_file_path, snaphu_exe_path, target_folder_snaphu_export_something_else)

Running SNAPHU command: "C:\Users\35196\.snap\auxdata\snaphu-v2.0.4_win64\bin\snaphu.exe" -f snaphu.conf Phase_ifg_IW1_VV_06Mar2021_10Feb2021.snaphu.img 20450
Working directory: C:\Users\35196\repos\Sentinel1_DEMgeneration\_results\_snaphu_export\goldstein


## 5.3. Snaphu Import

Just change the name after the unwrapping process (a file with a name similar to this one ```UnwPhase_ifg_IW1_VV_04Apr2025_16Apr2025.snaphu.img```

In [7]:
unwrapped_phase = snaphu_import(read_product(r'_results\goldstein.dim'), read_product(r'_results\_snaphu_export\goldstein\UnwPhase_ifg_IW1_VV_06Mar2021_10Feb2021.snaphu.img'))

SNAPHU importing...
SNAPHU imported...


In [8]:
write(unwrapped_phase, r'_results\unwrapped_phase', 'BEAM-DIMAP')

Writing the product...
Product written!


# 6. Back to porcessing workflow

## 6.1. Phase to Elevation

In [28]:
phase_elevation = phase_to_elevation(read_product(r'_results\unwrapped_phase.dim'), DEM)

Turning Phase to Elevation...
Phase to Elevation applied!


In [ ]:
write(phase_elevation, r'_results\phase_to_elevation', 'BEAM-DIMAP')

Writing the product...


## 6.2. Terrain Correction

In [30]:
terrain_correction = terrain_correction(read_product(r'_results\phase_to_elevation.dim'), DEM)

In [8]:
write(terrain_correction, r'_results\terrain_correction', 'BEAM-DIMAP')

Writing the product...


RuntimeError: no matching Java method overloads found

## 6.3. Subset

In [5]:
subset = subset(read_product(r'_results\terrain_correction.dim'), aoi)

Subsetting...
Subset applied!


In [6]:
write(subset, r'_results\subset', 'BEAM-DIMAP')

Writing the product...
Product written!


## 6.4. Export in GeoTIFF

In [7]:
write(subset, r'_results\dem', "GeoTiff")

Writing the product...
Product written!
